In [1]:
import os
import pynvml
from vllm import LLM, SamplingParams
from vllm.steer_vectors.request import SteerVectorRequest, VectorConfig
from transformers import AutoTokenizer


INFO 07-24 09:02:26 [__init__.py:244] Automatically detected platform cuda.


In [2]:


# 设置环境变量
os.environ["VLLM_USE_V1"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
# 模型路径
model_path = "/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/"

# 初始化tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 初始化LLM
llm = LLM(
    model=model_path,
    enable_steer_vector=True,
    enforce_eager=True,
    tensor_parallel_size=1
)

INFO 07-24 09:02:41 [config.py:841] This model supports multiple tasks: {'embed', 'generate', 'reward', 'classify'}. Defaulting to 'generate'.
INFO 07-24 09:02:41 [config.py:1472] Using max model len 32768
WARNING 07-24 09:02:42 [cuda.py:102] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 07-24 09:02:42 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7503+gfc71f0f) with config: model='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_ca

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 07-24 09:02:45 [default_loader.py:272] Loading weights took 0.99 seconds
INFO 07-24 09:02:45 [model_runner.py:1255] Model loading took 2.8876 GiB and 1.159209 seconds
INFO 07-24 09:02:47 [worker.py:295] Memory profiling takes 1.44 seconds
INFO 07-24 09:02:47 [worker.py:295] the current vLLM instance can use total_gpu_memory (47.44GiB) x gpu_memory_utilization (0.90) = 42.69GiB
INFO 07-24 09:02:47 [worker.py:295] model weights take 2.89GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 2.02GiB; the rest of the memory reserved for KV Cache is 37.72GiB.
INFO 07-24 09:02:47 [executor_base.py:115] # cuda blocks: 88295, # CPU blocks: 9362
INFO 07-24 09:02:47 [executor_base.py:120] Maximum concurrency for 32768 tokens per request: 43.11x
INFO 07-24 09:02:51 [llm_engine.py:430] init engine (profile, create kv cache, warmup model) took 5.86 seconds


In [ ]:
# Create a test prompt for a benign question (should get normal response)
example = "<|im_start|>user\nAnalyze the sentiment of the following movie review template (focus on ADJECTIVE and VERB positions):\nI believed this movie was ABYSMAL, I DETESTED it. Conclusion: This movie is unbearable.<|im_end|>\n<|im_start|>assistant\n"
# Generate baseline response without any steering
example_answers = llm.generate(
    example,
    SamplingParams(
        temperature=0,         # Deterministic generation
        max_tokens=128,        # Limit response length
        skip_special_tokens=False,  # Preserve special tokens
    ),
)

# Display the original model response (without steering)
print("=====Baseline=====")
print(example_answers[0].outputs[0].text)

# Configure steering vectors to trigger refusal behavior

# Target all layers of the model
layers = list(range(0,28))
scale = 2
# Create steering vector configuration
sv_request = SteerVectorRequest(
    # Basic configuration
    steer_vector_name="represent_sentiment",
    steer_vector_id=1,
    
    # Apply four steering vectors at different token positions
    vector_configs=[
        # Vector for token position 3
        VectorConfig(
            path="diffmean_3.gguf",      # The direction vector file
            scale=scale,                   # Strength of steering (positive reinforces refusal)
            target_layers=layers,        # Apply to all model layers
            prefill_trigger_positions=[3],  # Position of token to modify
            algorithm="direct",          # Direct vector application
            normalize=False              # Don't normalize vectors
        ),
        
        # Vector for token position 8
        VectorConfig(
            path="diffmean_8.gguf",
            scale=scale,
            target_layers=layers,
            prefill_trigger_positions=[8],
            algorithm="direct",
            normalize=False
        ),
        
        # Vector for token position 13
        VectorConfig(
            path="diffmean_13.gguf",
            scale=scale,
            target_layers=layers,
            prefill_trigger_positions=[13],
            algorithm="direct",
            normalize=False
        ),

        # Vector for token position 22
        VectorConfig(
            path="diffmean_22.gguf",
            scale=scale,
            target_layers=layers,
            prefill_trigger_positions=[22],
            algorithm="direct",
            normalize=False
        ),
        # Vector for token position 26 
        VectorConfig(
            path="diffmean_26.gguf",
            scale=scale,
            target_layers=layers,
            prefill_trigger_positions=[26],
            algorithm="direct",
            normalize=False
        ),
        # Vector for token position 29
        VectorConfig(
            path="diffmean_29.gguf",
            scale=scale,
            target_layers=layers,
            prefill_trigger_positions=[29],
            algorithm="direct",
            normalize=False
        ),
        # Vector for token position 41
        VectorConfig(
            path="diffmean_41.gguf",
            scale=scale,
            target_layers=layers,
            prefill_trigger_positions=[41],
            algorithm="direct",
            normalize=False
        ),
    ], 
    
    # Additional parameters
    debug=False,                      # Don't output debug info
    conflict_resolution="sequential"   # Apply vectors in sequence
)
# Generate response using refusal direction steering
# This should cause the model to refuse even benign requests
output = llm.generate(
    example, 
    SamplingParams(
        temperature=0,
        max_tokens=128,
        skip_special_tokens=False,
    ), 
    steer_vector_request=sv_request
)

# Display the steered response (should show refusal behavior)
print("=====Represent Sentiment Steered=====")
print(output[0].outputs[0].text)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

=====Baseline=====
The sentiment of the movie review template is negative. The adjectives "ABYSMAL" and "DETESTED" indicate strong disapproval, and the verb "is" in the conclusion further emphasizes the negative sentiment.


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

=====Refusal Direction Steered=====
The sentiment of the movie review template is negative. The adjectives "ABYSMAL" and "DETESTED" indicate strong disapproval, and the verb "is" in the conclusion further emphasizes the negative sentiment.


In [ ]:
# Create steering vector configuration
sv_request = SteerVectorRequest(
    # Basic configuration
    steer_vector_name="represent_sentiment",
    steer_vector_id=1,
    
    # Apply four steering vectors at different token positions
    vector_configs=[
        # Vector for token position 3
        VectorConfig(
            path="pca_3.gguf",      # The direction vector file
            scale=scale,                   # Strength of steering (positive reinforces refusal)
            target_layers=layers,        # Apply to all model layers
            prefill_trigger_positions=[3],  # Position of token to modify
            algorithm="direct",          # Direct vector application
            normalize=False              # Don't normalize vectors
        ),
        
        # Vector for token position 8
        VectorConfig(
            path="pca_8.gguf",
            scale=scale,
            target_layers=layers,
            prefill_trigger_positions=[8],
            algorithm="direct",
            normalize=False
        ),
        
        # Vector for token position 13
        VectorConfig(
            path="pca_13.gguf",
            scale=scale,
            target_layers=layers,
            prefill_trigger_positions=[13],
            algorithm="direct",
            normalize=False
        ),

        # Vector for token position 22
        VectorConfig(
            path="pca_22.gguf",
            scale=scale,
            target_layers=layers,
            prefill_trigger_positions=[22],
            algorithm="direct",
            normalize=False
        ),
        # Vector for token position 26 
        VectorConfig(
            path="pca_26.gguf",
            scale=scale,
            target_layers=layers,
            prefill_trigger_positions=[26],
            algorithm="direct",
            normalize=False
        ),
        # Vector for token position 29
        VectorConfig(
            path="pca_29.gguf",
            scale=scale,
            target_layers=layers,
            prefill_trigger_positions=[29],
            algorithm="direct",
            normalize=False
        ),
        # Vector for token position 41
        VectorConfig(
            path="pca_41.gguf",
            scale=scale,
            target_layers=layers,
            prefill_trigger_positions=[41],
            algorithm="direct",
            normalize=False
        ),
    ], 
    
    # Additional parameters
    debug=False,                      # Don't output debug info
    conflict_resolution="sequential"   # Apply vectors in sequence
)
# Generate response using refusal direction steering
# This should cause the model to refuse even benign requests
output = llm.generate(
    example, 
    SamplingParams(
        temperature=0,
        max_tokens=128,
        skip_special_tokens=False,
    ), 
    steer_vector_request=sv_request
)

# Display the steered response (should show refusal behavior)
print("=====Represent Sentiment Steered=====")
print(output[0].outputs[0].text)